In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Reviewing the Outages RAW Data
SELECT * 
FROM 
    OUTAGES_RAW_COORDINATES 
WHERE 
    1 = 1
    AND INCIDENT_TEXT ILIKE '%tree%';

In [ ]:
-- Categories table. 
SELECT * 
FROM OUTAGES_CATEGORIES;

In [ ]:
-- Converting categories into a single VARIANT
CREATE OR REPLACE TABLE OUTAGES_CATEGORIES_ARRAY AS 
SELECT OBJECT_AGG(CATEGORY, KEYPHRASES) AS CATEGORY_PAYLOAD
FROM (
    SELECT CATEGORY, ARRAY_AGG(KEYPHRASE) AS KEYPHRASES
    FROM OUTAGES_CATEGORIES
    GROUP BY CATEGORY
);

In [ ]:
SELECT * FROM OUTAGES_CATEGORIES_ARRAY;

SELECT TO_CHAR(CATEGORY_PAYLOAD) FROM OUTAGES_CATEGORIES_ARRAY;

In [ ]:
-- SETTING VARIABLES TO WORK WITH THE LLM
SET CATEGORY_PROMPT = $$Classify the text provided into one or multiple of these categories. ONLY show categories matching the keyphrase and nothing else. Don't elaborate. If there is more than 1 category, ALWAYS show them comma separated. If no matching categories No Match: $$;

SET CLASSIFYING_ISSUE_PROMPT = $$ text provided: $$;

SET COUNT_PART_PROMPT = $$Identify parts and its corresponding count from the [Provided Text]. Enforce the format below. Return as a JSON object. If no match, return 0. Stay on task and only provide the Expected Format:
{
"Poles": 2 
"Transformer": 3
"Line": 1
}

[Provided Text]: 
$$;

SET MODEL1 = 'llama3.1-70b';
SET MODEL2 = 'llama3.1-405b';
SET MODEL3 = 'mistral-7b';

In [ ]:
-- RUNNING BATCH LLM INFERENCE
CREATE OR REPLACE TABLE OUTAGES_GENAI_CATEGORIES AS
SELECT 
    INCIDENT_NUMBER, 
    INCIDENT_TEXT,
    LATITUDE,
    LONGITUDE,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3.1-405b', 
        CONCAT($$
            {
              "task": "Classify the incident into one or more of the predefined categories based on the provided keyphrases. Return ONLY the matching category names as a comma-separated list. If no category matches, return 'No Match'. Do not include any explanation, introduction, or additional text outside of the required response.",
              "categories": $$, (SELECT TO_CHAR(CATEGORY_PAYLOAD) FROM OUTAGES_CATEGORIES_ARRAY), $$
              "note": "Output format: 'Category1, Category2, ...'. If no match, output: 'No Match'.",
              "incident": $$, '{', INCIDENT_TEXT, $$
            }
        $$) 
    ) AS classification_result_llama3_1_405b,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3.1-405b', 
        CONCAT($$
            {
              "task": "Identify parts and their corresponding count from the provided incident text. Return ONLY the JSON object in the exact expected format below. Do not include any explanation, introduction, or additional text.",
              "expected_format": {
                "Poles": 0,
                "Transformer": 0,
                "Line": 0
              },
              "note": "Do not include any text outside the JSON object.",
              "incident": $$, '{', INCIDENT_TEXT, $$
            }
        $$) 
    ) AS part_count_result_llama3_1_405b
FROM 
    OUTAGES_RAW_COORDINATES 
WHERE 
    INCIDENT_TEXT != '';

In [ ]:
SELECT * FROM OUTAGES_GENAI_CATEGORIES;

In [ ]:
-- Counting the parts per category
CREATE OR REPLACE TABLE OUTAGES_GENAI_CATEGORIES_COUNT AS
WITH flattened_categories AS (
    SELECT 
        INCIDENT_NUMBER, 
        INCIDENT_TEXT, 
        LATITUDE,
        LONGITUDE,
        CLASSIFICATION_RESULT_LLAMA3_1_405B,
        TRIM(value::STRING) AS category
    FROM 
        OUTAGES_GENAI_CATEGORIES  
    , LATERAL FLATTEN(input => SPLIT(classification_result_llama3_1_405b, ','))  -- Flattening the categories for 405b
)
SELECT 
    LATITUDE,
    LONGITUDE,
    category, 
    COUNT(*) AS occurrence_count
FROM 
    flattened_categories
GROUP BY 
    LATITUDE,
    LONGITUDE,
    category
ORDER BY 
    occurrence_count DESC;

In [ ]:
-- SHOWING THE COUNTS PER CATEGORY
SELECT * 
FROM   OUTAGES_GENAI_CATEGORIES_COUNT;

## Create Cortex Search service 

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE HURRICANE_CORTEX_SEARCH 
ON INCIDENT_TEXT 
ATTRIBUTES
	INCIDENT_NUMBER 
WAREHOUSE = DEMO_WH 
TARGET_LAG = '365 days' 
AS (
	SELECT
		INCIDENT_TEXT,CLASSIFICATION_RESULT_LLAMA3_1_405B,INCIDENT_NUMBER
	FROM 
        CLASSIFICATION_DB.HURRICANE_SCHEMA.OUTAGES_GENAI_CATEGORIES
);